Test NB- Capstone Winter 2022- K.Matulay

In [2]:
# imports from Python standard library
import re
import string
from collections import Counter

# imports requiring installation
#   connection to Google Cloud Storage
#from google.cloud import storage            # pip install google-cloud-storage
#from google.oauth2 import service_account   # pip install google-auth

#  data science packages
import matplotlib.pyplot as plt             # pip install matplotlib
import nltk                                 # pip install nltk
import numpy as np                          # pip install numpy
import pandas as pd                         # pip install pandas
import seaborn as sns                       # pip install seaborn
#import squarify                             # pip install squarify

from nltk.tokenize import TweetTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
#from wordcloud import WordCloud

In [3]:
import os
import pyarrow

In [4]:
os.chdir('/Users/katymatulay/Documents/Drexel - Grad School/07 Fall 2022/DSCI591/Capstone/data')
merged_df = pd.read_parquet("../data/merged_df_preprocessed.parquet.snappy")

In [5]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3624894 entries, 0 to 3624893
Data columns (total 18 columns):
 #   Column              Dtype              
---  ------              -----              
 0   external_author_id  string             
 1   author              string             
 2   content             string             
 3   region              string             
 4   language            string             
 5   following           uint64             
 6   followers           uint64             
 7   updates             uint64             
 8   post_type           string             
 9   is_retweet          float64            
 10  account_category    string             
 11  tweet_id            string             
 12  tco1_step1          string             
 13  data_source         string             
 14  has_url             int64              
 15  emoji_text          object             
 16  emoji_count         int64              
 17  publish_date        datetim

In [6]:
merged_df.head(5)

,external_author_id,author,content,region,language,following,followers,updates,post_type,is_retweet,account_category,tweet_id,tco1_step1,data_source,has_url,emoji_text,emoji_count,publish_date
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,1052,9636,253,<NA>,0.0,RightTroll,914580356430536707,https://twitter.com/10_gop/status/914580356430...,Troll,1,[],0,2017-10-01 19:58:00+00:00
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,1054,9637,254,<NA>,0.0,RightTroll,914621840496189440,https://twitter.com/damienwoody/status/9145685...,Troll,1,[],0,2017-10-01 22:43:00+00:00
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,1054,9637,255,RETWEET,1.0,RightTroll,914623490375979008,https://twitter.com/10_gop/status/913231923715...,Troll,1,[],0,2017-10-01 22:50:00+00:00
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,1062,9642,256,<NA>,0.0,RightTroll,914639143690555392,https://twitter.com/10_gop/status/914639143690...,Troll,1,[],0,2017-10-01 23:52:00+00:00
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,1050,9645,246,RETWEET,1.0,RightTroll,914312219952861184,https://twitter.com/realDonaldTrump/status/914...,Troll,1,[flag: United States],1,2017-10-01 02:13:00+00:00


Create label: 1= Fake, 0= Verified/Real

In [7]:
label_map = {
    'Troll': 1, 
    'verified_user': 0, 
    'verified_random': 0
    }

merged_df['label'] = merged_df['data_source'].map(label_map)

merged_df['label'].value_counts(dropna=False)

1    2116866
0    1508028
Name: label, dtype: int64